In [ ]:
import sys

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
sys.path.append(repo_root)  # go to parent dir

from spf.dataset.spf_dataset import SessionsDatasetReal

In [ ]:
from spf.utils import dotdict

args = dotdict(
    {
        "carrier_frequency": 2.4e9,
        "signal_frequency": 100e3,
        "sampling_frequency": 10e6,
        "array_type": "linear",  # "circular"],
        "elements": 2,
        "random_silence": False,
        "detector_noise": 0.0,  # 1e-4,
        "random_emitter_timing": False,
        "sources": 1,
        "seed": 0,
        "beam_former_spacing": 64 + 1,
        "width": 128,
        "detector_trajectory": "bounce",
        "detector_speed": 0.0,
        "source_speed": 10.0,
        "sigma_noise": 0.0,  # 1.0,
        "time_steps": 4096,
        "time_interval": 0.3,
        "readings_per_snapshot": 3,
        "sessions": 4096,
        "reference": False,
        "cpus": 8,
        "live": True,
        "profile": False,
        "fixed_detector": None,  #
    }
)

In [ ]:
from spf.dataset.spf_generate import generate_session_and_dump
import tempfile
from compress_pickle import dump

tmpdir = tempfile.TemporaryDirectory()
args.output = tmpdir.name
dump(args, "/".join([args.output, "args.pkl"]), compression="lzma")
# result = [ generate_session_and_dump((args, session_idx)) for session_idx in range(args.sessions) ]

In [ ]:
from spf.dataset.spf_dataset import SessionsDatasetSimulated

ds = SessionsDatasetSimulated(root_dir=tmpdir.name, snapshots_per_session=1024)

In [ ]:
# pick a session from the dataset
session_idx = 1000
session = ds[session_idx]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spf.rf import get_peaks_for_2rx

idx = 490
bf_out = session["beam_former_outputs_at_t"][idx]
peaks = get_peaks_for_2rx(bf_out)
plt.plot(bf_out, color="red", label="beamformer")
for peak in peaks:
    plt.axvline(x=peak, label="Peak")
plt.legend()

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(16, 8))

axs[0, 0].invert_yaxis()  # y goes down
axs[0, 0].invert_xaxis()  # x goes left
axs[0, 0].set_xlabel("X position")
axs[0, 0].set_ylabel("Y position")
axs[0, 0].plot(
    session["source_positions_at_t"][:, 0, 0], session["source_positions_at_t"][:, 0, 1]
)
axs[0, 0].set_title("2D Position (x/y)")

axs[1, 0].plot(session["source_positions_at_t"][:, 0, 0], label="X")
axs[1, 0].plot(session["source_positions_at_t"][:, 0, 1], label="Y")
axs[1, 0].legend()
axs[1, 0].set_title("Position (x/y) vs time")
axs[1, 0].set_xlabel("Time")

axs[1, 1].plot(session["detector_orientation_at_t"] + session["source_theta_at_t"])
axs[1, 1].set_ylabel("Theta")
axs[1, 1].set_xlabel("Time")
axs[1, 1].set_title("Ground truth theta")

# plot the beamformer output
peaks_at_t = np.array(
    [get_peaks_for_2rx(bf_out) for bf_out in session["beam_former_outputs_at_t"]]
)
peaks_at_t_in_radians = 2 * (peaks_at_t / args.beam_former_spacing - 0.5) * np.pi
peaks_at_t_in_radians_adjusted = (
    peaks_at_t_in_radians + 20 * session["detector_orientation_at_t"]
)  # % (np.pi*2)

axs[0, 1].plot(peaks_at_t_in_radians_adjusted[:, 0])
axs[0, 1].plot(peaks_at_t_in_radians_adjusted[:, 1])
axs[0, 1].set_xlabel("Time")
axs[0, 1].set_ylabel("Theta")

f.tight_layout()

In [ ]:
peaks_at_t = np.array(
    [get_peaks_for_2rx(bf_out) for bf_out in session["beam_former_outputs_at_t"]]
)
peaks_at_t_in_radians = 2 * (peaks_at_t / args.beam_former_spacing - 0.5) * np.pi
peaks_at_t_in_radians_adjusted = (
    peaks_at_t_in_radians + session["detector_orientation_at_t"]
)  # % (np.pi*2)
peaks_at_t_in_radians_adjusted

In [ ]:
ground_truth = session["detector_orientation_at_t"] + session["source_theta_at_t"]

In [ ]:
np.abs(peaks_at_t_in_radians_adjusted - ground_truth).min(axis=1).mean()